In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [11]:
key_path = '../service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [12]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

# SELECTED CATEGORY

In [19]:
df_cat = pd.read_csv('../result/category.csv')

print(df_cat.info())
df_cat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5 non-null      int64 
 1   category  5 non-null      object
dtypes: int64(1), object(1)
memory usage: 212.0+ bytes
None


,id,category
0,0,Accessories
1,1,Apparel
2,2,Electronics
3,3,Office
4,4,Shop by Brand


In [20]:
# Get the schema of the result
schema = [
    bigquery.SchemaField("id", "INTEGER"),
    bigquery.SchemaField("category", "STRING"),
]

# Define the new table reference
project_id = credentials.project_id
dataset_id = 'dummy'
table_id = 'category'
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

# Delete the existing destination table if it exists
try:
    client.delete_table(table_ref)
    print(f"Deleted table {table_ref}")
except Exception as e:
    print(f"Table {table_ref} does not exist: {e}")
        
# Create a new table with the schema
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request


# Load DataFrame to BigQuery table
job = client.load_table_from_dataframe(df_cat, table_ref)

# Wait for the load job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Table gentle-keyword-423715-j0.dummy.category does not exist: 404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/gentle-keyword-423715-j0/datasets/dummy/tables/category?prettyPrint=false: Not found: Table gentle-keyword-423715-j0:dummy.category
Loaded 5 rows into dummy:category.


# TOP PRODUCT

In [8]:
df_prod = pd.read_csv('../result/top_product.csv')


print(df_prod.info())
df_prod.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 16 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   category                               75 non-null     object
 1   product                                75 non-null     object
 2   visits                                 75 non-null     int64 
 3   time_on_site                           75 non-null     int64 
 4   transactions                           75 non-null     int64 
 5   revenue                                75 non-null     int64 
 6   medium_none                            75 non-null     int64 
 7   medium_affiliate                       75 non-null     int64 
 8   medium_cpc                             75 non-null     int64 
 9   medium_cpm                             75 non-null     int64 
 10  medium_organic                         75 non-null     int64 
 11  medium_referral      

,category,product,visits,time_on_site,transactions,revenue,medium_none,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Accessories,8 pc Android Sticker Sheet,366,373644,109,24754460000,228,2,11,2,105,18,0,196,0,170
1,Accessories,Android Sticker Sheet Ultra Removable,465,420577,91,10802510000,220,2,26,7,201,9,0,253,0,212
2,Accessories,Basecamp Explorer Powerbank Flashlight,81,97267,13,3518450000,40,0,0,0,39,2,0,47,0,34
3,Accessories,Google Car Clip Phone Holder,123,98510,14,668220000,78,0,12,6,24,3,0,73,0,50
4,Accessories,Google Device Stand,20,18840,4,78280000,7,0,2,4,7,0,0,10,0,10


In [18]:
# Get the schema of the result
schema = [
    bigquery.SchemaField("category", "STRING"),
    bigquery.SchemaField("product", "STRING"),
    bigquery.SchemaField("visits", "INTEGER"),
    bigquery.SchemaField("time_on_site", "INTEGER"),
    bigquery.SchemaField("transactions", "INTEGER"),
    bigquery.SchemaField("revenue", "INTEGER"),
    bigquery.SchemaField("medium_none", "INTEGER"),
    bigquery.SchemaField("medium_affiliate", "INTEGER"),
    bigquery.SchemaField("medium_cpc", "INTEGER"),
    bigquery.SchemaField("medium_cpm", "INTEGER"),
    bigquery.SchemaField("medium_organic", "INTEGER"),
    bigquery.SchemaField("medium_referral", "INTEGER"),
    bigquery.SchemaField("action_Check_out", "INTEGER"),
    bigquery.SchemaField("action_Click_through_of_product_lists", "INTEGER"),
    bigquery.SchemaField("action_Completed_purchase", "INTEGER"),
    bigquery.SchemaField("action_Product_detail_views", "INTEGER"),
]

# Define the new table reference
project_id = credentials.project_id
dataset_id = 'dummy'
table_id = 'top-products'
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

# Delete the existing destination table if it exists
try:
    client.delete_table(table_ref)
    print(f"Deleted table {table_ref}")
except Exception as e:
    print(f"Table {table_ref} does not exist: {e}")
        
# Create a new table with the schema
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request


# Load DataFrame to BigQuery table
job = client.load_table_from_dataframe(df_prod, table_ref)

# Wait for the load job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Deleted table gentle-keyword-423715-j0.dummy.top-products
Loaded 75 rows into dummy:top-products.


# Predictions

In [41]:
df = pd.DataFrame()
categories = pd.read_csv('../result/category.csv')['category'].values
for cat in categories:
    df_exp = pd.read_csv(f'../result/prediction-revenue-{cat}.csv')
    df = pd.concat([df, df_exp])

df['date'] = pd.to_datetime(df['date'])
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 119
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               600 non-null    datetime64[ns]
 1   transactions_sum   445 non-null    float64       
 2   transactions_pred  600 non-null    int64         
 3   category           600 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 23.4+ KB
None


,date,transactions_sum,transactions_pred,category
0,2017-05-04,24.0,23,Accessories
1,2017-05-05,23.0,23,Accessories
2,2017-05-06,12.0,11,Accessories
3,2017-05-07,0.0,0,Accessories
4,2017-05-08,30.0,29,Accessories


In [42]:
# Get the schema of the result
schema = [
    bigquery.SchemaField("date", "DATE"),
    bigquery.SchemaField("transactions_sum", "INTEGER"),
    bigquery.SchemaField("transactions_pred", "INTEGER"),
    bigquery.SchemaField("category", "STRING"),
]

# Define the new table reference
project_id = credentials.project_id
dataset_id = 'dummy'
name_table = cat.replace("'", "").replace(" ", "")
table_id = f'prediction-revenue'
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

# Delete the existing destination table if it exists
try:
    client.delete_table(table_ref)
    print(f"Deleted table {table_ref}")
except Exception as e:
    print(f"Table {table_ref} does not exist: {e}")

# Create a new table with the schema
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request


# Load DataFrame to BigQuery table
job = client.load_table_from_dataframe(df, table_ref)

# Wait for the load job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Deleted table gentle-keyword-423715-j0.dummy.prediction-revenue
Loaded 600 rows into dummy:prediction-revenue.
